In [3]:
import pandas as pd
from google.cloud import bigquery
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

True

In [86]:
client = bigquery.Client(location="EU")
all = []
for table in ["hours_25", "hours_24_25", "hours_23_24", "hours_22_23"]:
    df = client.query(f"""
        SELECT *
        FROM `registrations.{table}`
    """).to_dataframe()
    df["date_completed"] = pd.to_datetime(df["date_completed"], utc=True)
    if table == "hours_25":
        df["season"] = "25/26"
    else:
        df["season"] = "/".join(table.split("_")[1:])
    all.append(df)

/Users/sigvardbratlie/Documents/Projects/genf/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/sigvardbratlie/Documents/Projects/genf/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/sigvardbratlie/Documents/Projects/genf/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/sigvardbratlie/Documents/Projects/genf/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [87]:
df = pd.concat(all)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8231 entries, 0 to 2374
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date_completed       8231 non-null   datetime64[ns, UTC]
 1   worker_name          8231 non-null   object             
 2   comments             3415 non-null   object             
 3   hours_worked         8216 non-null   float64            
 4   number_of_units      3415 non-null   Int64              
 5   date_of_birth        1169 non-null   datetime64[us, UTC]
 6   bank_account_number  3415 non-null   Int64              
 7   email                3415 non-null   object             
 8   role                 8231 non-null   object             
 9   work_type            8229 non-null   object             
 10  season               8231 non-null   object             
dtypes: Int64(2), datetime64[ns, UTC](1), datetime64[us, UTC](1), float64(1), object(6)
memo

In [88]:
df.head()

,date_completed,worker_name,comments,hours_worked,number_of_units,date_of_birth,bank_account_number,email,role,work_type,season
0,2025-09-06 00:00:00+00:00,Alexander,…,3.0,0,2012-10-29 00:00:00+00:00,36264854843,alexnilsen2012@gmail.com,genf,bccof_utomhus,25/26
1,2025-12-18 00:00:00+00:00,Alexander,mentor: kristian,2.0,0,2012-10-29 00:00:00+00:00,36264854843,alexnilsen2012@gmail.com,genf,bccof_vask,25/26
2,2025-11-22 00:00:00+00:00,Alexander,(43 antall) mentor alex og sigvard,4.0,43,2012-10-29 00:00:00+00:00,36264854843,alexnilsen2012@gmail.com,genf,glenne_vedpakking,25/26
3,2025-11-09 00:00:00+00:00,Alexander,mentor: martin og fredrik,2.5,0,2012-10-29 00:00:00+00:00,36264854843,alexnilsen2012@gmail.com,genf,bccof_rigg,25/26
4,2025-10-26 00:00:00+00:00,Amilie,Judith grimes,2.0,0,2012-02-29 00:00:00+00:00,92303150465,amiliesavannah12@gmail.com,genf,bccof_vask,25/26


In [89]:
df["date_completed"].min(), df["date_completed"].max(), df["hours_worked"].sum()

(Timestamp('2022-08-02 00:00:00+0000', tz='UTC'),
 Timestamp('2025-12-25 00:00:00+0000', tz='UTC'),
 np.float64(27133.449999988134))

In [90]:
client.load_table_from_dataframe(df, "registrations.hours_22_25",job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")).result()

/Users/sigvardbratlie/Documents/Projects/genf/.venv/lib/python3.13/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=genf-446213, location=EU, id=03da8d5d-354f-4621-8d82-ce1182c1348e>